# Demo para analizar y estudiar las imágenes disponibles


In [1]:
#@title Librerías a usar

import tensorflow_hub as hub
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
from tensorflow.keras import layers

import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os

from PIL import Image, ExifTags, ImageStat

from sklearn import tree
from sklearn.neighbors import KernelDensity
from sklearn.cluster import KMeans

import ipywidgets as widgets
from ipywidgets import Box, Layout
from IPython.display import clear_output

print("Librerías cargadas")

Librerías cargadas


In [2]:
#@title Funciones Auxiliares

# variables auxiliares
atributo_clase = ""

# configura para que muestre todas las columnas y filas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# devuelve listas de columnas numéricas y no numéricas
def devolNombreColumnas(ndf):
  colValues = []
  colNoValues = []
  for col in ndf.columns:
    if ndf[col].dtypes in ("object", "bool"):
      colNoValues.append( col )
    else:
      colValues.append( col )
  return colValues, colNoValues

# función auxiliar para separar datos de entrada y de salida
def separarDatosXY(ndf, atributo_clase="", xSoloNros=True):
  # hace una copia auxiliar del data frame
  cdf = ndf.copy()
  # saca el atributo clase (OPCIONAL)
  if atributo_clase == "":
    Y = []
  else:
    # datos atributo clase
    Y = np.array( cdf.pop(atributo_clase).fillna("-NAN-") )
  if xSoloNros:
    # se queda sólo con los atributos numéricos (OPCIONAL)
    for col in cdf.columns:
      if cdf[col].dtypes == "object":
          cdf.pop( col )
  # datos de entrada
  X = np.array(cdf.fillna(0.001))
  return X, Y, np.array(cdf.columns)

def convColsNumericas(ndf, atributos_no_convertir = []):
  # hace una copia auxiliar del data frame
  cdf = ndf.copy()
  # convierte todas las no numéricas a numéricas (OPCIONAL)
  for col in cdf.columns:
    if col not in atributos_no_convertir:
      if cdf[col].dtypes == "object":
        # genera diccionario de valores
        valores = cdf[col].unique()
        diccValores = dict(zip(valores, range(len(valores))))
        # realiza el reemplazo
        cdf[col] = cdf[col].map(lambda s: diccValores.get(s) if s in diccValores else s)
  return cdf

# función auxiliar
def generar_estadisticas_detalladas(orDF, titulo=""):
  # título
  print("\n", titulo, ": ")
  # obtiene las estadísticas generales
  estDF = orDF.describe().transpose()
  #  genera y formatea las estadísticas
  if "min" in estDF and "max" in estDF:
    rangoValores = "[ " + estDF["min"].apply('{:.2f}'.format) + " ; " + estDF["max"].apply('{:.2f}'.format) + " ]"
  else:
    rangoValores = estDF["unique"].fillna(0.0).apply('{:.0f}'.format)
  rangoValores.name = "Rango Valores"
  # para campos no numéricos muestra las cantidades por valor
  for col in orDF.columns:
    if orDF[col].dtypes in ("object", "bool"):
      auxStr = str( orDF[col].value_counts() ).replace("\n", " ; ")
      if (auxStr.index("Name")-3) > 0:
        # saca lo del final porque no sirve
        auxStr = auxStr[:auxStr.index("Name")-3]
      rangoValores[col] = "{ " + auxStr + " }"
  if "mean" in estDF and "std" in estDF:
    promValores = estDF["mean"].fillna(0.0).apply('{:.3f}'.format) + " ± " + estDF["std"].fillna(0.0).apply('{:.3f}'.format)
  else:
    promValores = estDF["count"].apply('{:.0f}'.format)
  promValores.name = "Promedio ± Desvío"
  # obtiene valores "ceros" y nulos
  zero_val = (orDF == 0.00).astype(int).sum(axis=0)
  zero_val.name = "¿Valores Ceros?"
  mis_val = orDF.isnull().sum()
  mis_val.name = "¿Valores Nulos?"
  # prepara la nueva tabla para mostrar
  nTable = pd.concat([orDF.dtypes, rangoValores, promValores, zero_val, mis_val], axis=1)
  nTable = nTable.rename( columns = {0: 'Tipo Valor',  1: 'Rango Valores', 2: 'Promedio ± Desvío', 3: '¿Valores Ceros?', 4: '¿Valores Nulos?' } )
  # muestra la nueva tabla
  pd.set_option('max_colwidth', None)
  display(nTable.fillna("-"))
  print("Tiene " + str(orDF.shape[1]) + " atributos y " + str(orDF.shape[0]) + " ejemplos.")
  print("\n")
  return

print("Funciones auxiliares definidas.")

Funciones auxiliares definidas.


# Carga de los Datos:

In [3]:
#@title Montar Google Drive { run: "auto" }

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#@markdown ### Directorio  en Google Drive:
path = 'gdrive/MyDrive/IA/demoML/imagenes/NUMEROS' #@param {type:"string"}

#@markdown ### Subdirectorios de imágenes:
path_entrenamiento = '/train'  #@param {type:"string"}
path_validacion = ''  #@param {type:"string"}
path_prueba = '/test'  #@param {type:"string"}

# función auxiliar
def controlarPath(paths_imagenes, path, subdir, tipoUso):
  if path != "" and subdir != "":
    pCompleto = path + subdir
    if os.path.isdir(pCompleto):
      print("Usando imágenes de " + tipoUso + ": ", pCompleto)
      paths_imagenes.append( (pCompleto, tipoUso) )
    else:
      print("Error: El directorio de " + tipoUso + " ", pCompleto, " no existe!!!")


# define path completo y verifica que existan
paths_imagenes = []
controlarPath(paths_imagenes, path, path_entrenamiento, "entrenamiento")
controlarPath(paths_imagenes, path, path_validacion, "validacion")
controlarPath(paths_imagenes, path, path_prueba, "prueba")


Mounted at /content/gdrive
Usando imágenes de entrenamiento:  gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train
Usando imágenes de prueba:  gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test


In [4]:
#@title Cargar información de las Imágenes

# parámetros de información a cargar
incluir_info_Estadisticas_Imagen = True #@param {type:"boolean"}
incluir_info_Extra_Imagen = False #@param {type:"boolean"}
incluir_info_EXIF_Imagen = False #@param {type:"boolean"}

# función auxiliar para agregar elemenos al diccionario
# si no sabemos si es lista o valor
# nota: lo hace recursivo para manejar datos anidados
def agregarInfoDic(dicc, clave, valor):
  if isinstance(valor, (list, tuple, np.ndarray, np.ndenumerate)):
    # si es lista, entonces agrega una columna por elemento de la lista
    i = 0
    while i < len(valor):
      #dicc[clave+"_"+str(i)] = valor[i]
      agregarInfoDic(dicc, clave+"_"+str(i), valor[i])
      i = i + 1
  elif isinstance(valor, dict):
    # si es un diccionario, entonces agrega una columna por elemento de la lista
    for k in valor.keys():
      #dicc[clave+"_"+str(k)] = valor[k]
      agregarInfoDic(dicc, clave+"_"+str(k), valor[k])
  elif isinstance(valor, bytes):
     # si es bytes, lo tranforma a ASCII
     dicc[clave] = str(valor.decode("utf-8", "ignore"))
  else:
    # sino agrega directo el valor
    dicc[clave] = valor

# define función para cargar información de las imágenes
def cargarInfoImagenes(imagPath, tipoUso, incluirInfoEstadImag=False, incluirInfoExtraImag=False, incluirInfoEXIF=False, stringDA='' ):
  print("\n")
  if imagPath is None or imagPath == "":
    print("Error: no se indica el path de las imágenes.")
    return []
  if tipoUso == "":
    print("> Procesando imágenes:")
  else:
    print("> Procesando imágenes de ", tipoUso, ":")
  # inicializa valores
  cantMostrar = 10
  infoImag = []
  all_dirs = os.listdir( imagPath )
  for each_dir in all_dirs:
    # controla que sea un subdirectorio
    auxiPath = imagPath + '/' + each_dir
    if os.path.isdir(auxiPath):
      # procesa por subdirectorio de CLASE
      imagFN  = os.listdir( auxiPath )
      clase = each_dir
      totalMostrar = len(imagFN)
    else:
      # procesa el archivo
      imagFN = [ each_dir ]
      auxiPath = imagPath + '/'
      clase = ""
      totalMostrar = -1
    # auxiliar para mosrar progreso
    cant = 0
    for each_imagFN in imagFN:
      if totalMostrar > 0:
        cant = cant + 1
        if ((cantMostrar>0) and ((cant%cantMostrar)==0)) or (cant==totalMostrar):
          print("  -- subdirectorio ", each_dir, ":  procesando ", cant, " imágenes de ", totalMostrar)
      # inicializa el diccionario
      dicInfo = {}
      # carga la información base
      if tipoUso != "":
        dicInfo["TipoUso"] = tipoUso
      dicInfo["Imagen"] = each_imagFN
      if clase != "":
        dicInfo["Clase"] = clase
      # abre la imagen y obtiene información general
      img = Image.open(auxiPath+"/"+each_imagFN)
      dicInfo["Img_Ancho"] = img.width
      dicInfo["Img_Alto"] = img.height
      dicInfo["Img_Modo"] = img.mode
      if incluirInfoEstadImag:
        # obtiene información estadistica de la imagen
        # si se quieren agregar más, ver https://pillow.readthedocs.io/en/stable/reference/ImageStat.html
        statImg = ImageStat.Stat(img)
        for b, c, m, s in zip(img.getbands(), statImg.count, statImg.mean, statImg.stddev):
            b = b.upper()
            if b == "B":
              banda = "Azul"
            elif b == "G":
              banda = "Verde"
            elif b == "R":
              banda = "Rojo"
            elif b == "A":
              banda = "Alfa"
            else:
              banda = b
            dicInfo["Pixeles_"+banda+"_Total"] = c
            dicInfo["Pixeles_"+banda+"_Promedio"] = m
            dicInfo["Pixeles_"+banda+"_Desvio"] = s
      if incluirInfoExtraImag:
        # obtiene información extra de la imagen
        for key in img.info.keys():
          #dicInfo[key] = img.info[key]
          if key != "exif":
            agregarInfoDic(dicInfo, "Img_"+key, img.info[key])
      if incluirInfoEXIF:
        #  obtiene la información EXIF de la imagen
        exif_data = img._getexif()
        if exif_data is None:
          dicInfo["TieneEXIF"] = False
        else:
          dicInfo["TieneEXIF"] = True
          if isinstance(exif_data, bytes):
              exif_data = exif_data.decode()
          for key, val in exif_data.items():
            if key in ExifTags.TAGS:
              strK = "EXIF_"+ExifTags.TAGS[key]
            elif key in ExifTags.GPSTAGS:
              strK = "EXIF_"+ExifTags.GPSTAGS[key]
            else:
              strK = "EXIF_"+str(key)
            #dicInfo[ExifTags.TAGS[key]] = val
            agregarInfoDic(dicInfo, strK, val)
      # obtiene información sobre path y tipo archivo
      dicInfo["Archivo_Path"] = auxiPath
      dicInfo["Archivo_Nombre"] = os.path.splitext(each_imagFN)[0]
      dicInfo["Archivo_Extension"] = os.path.splitext(each_imagFN)[1].upper()
      dicInfo["Archivo_TamBytes"] =os.path.getsize(auxiPath+"/"+each_imagFN)
      if stringDA != '':
        # determina si es una imagen generada por data augmentation
        dicInfo["Archivo_EsDA"] = (each_imagFN[:2] == stringDA)
      # agrega la información a la lista
      infoImag.append( dicInfo )
  print("  == se cargan ", len(infoImag), "imágenes ==")
  return infoImag

# carga las imagenes de acuerdo a los path definidos antes
todaInfo = []
for p in paths_imagenes:
  auxInfo = cargarInfoImagenes(p[0],
                                       tipoUso = p[1],
                                       incluirInfoEstadImag = incluir_info_Estadisticas_Imagen,
                                       incluirInfoExtraImag = incluir_info_Extra_Imagen,
                                       incluirInfoEXIF = incluir_info_EXIF_Imagen,
                                       stringDA = 'da')
  todaInfo.extend( auxInfo )

# se genera el data frame con los datos cargados
if len(todaInfo) == 0:
  print("\nNo se ha podido cargar la información de las imágenes!")
  ndf = None
else:
  ndf = pd.DataFrame( todaInfo )
  print("\n> Información cargada de las imágenes: ")
  display( ndf )



> Procesando imágenes de  entrenamiento :
  -- subdirectorio  0 :  procesando  10  imágenes de  24
  -- subdirectorio  0 :  procesando  20  imágenes de  24
  -- subdirectorio  0 :  procesando  24  imágenes de  24
  -- subdirectorio  4 :  procesando  10  imágenes de  24
  -- subdirectorio  4 :  procesando  20  imágenes de  24
  -- subdirectorio  4 :  procesando  24  imágenes de  24
  -- subdirectorio  5 :  procesando  10  imágenes de  24
  -- subdirectorio  5 :  procesando  20  imágenes de  24
  -- subdirectorio  5 :  procesando  24  imágenes de  24
  -- subdirectorio  6 :  procesando  10  imágenes de  24
  -- subdirectorio  6 :  procesando  20  imágenes de  24
  -- subdirectorio  6 :  procesando  24  imágenes de  24
  -- subdirectorio  8 :  procesando  10  imágenes de  24
  -- subdirectorio  8 :  procesando  20  imágenes de  24
  -- subdirectorio  8 :  procesando  24  imágenes de  24
  -- subdirectorio  1 :  procesando  10  imágenes de  24
  -- subdirectorio  1 :  procesando  20  imá

,TipoUso,Imagen,Clase,Img_Ancho,Img_Alto,Img_Modo,Pixeles_L_Total,Pixeles_L_Promedio,Pixeles_L_Desvio,Archivo_Path,Archivo_Nombre,Archivo_Extension,Archivo_TamBytes,Archivo_EsDA
0,entrenamiento,img001-00052.png,0,128,128,L,16384,197.708801,104.907928,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00052,.PNG,1066,False
1,entrenamiento,img001-00077.png,0,128,128,L,16384,158.532166,122.573417,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00077,.PNG,1059,False
2,entrenamiento,img001-00090.png,0,128,128,L,16384,194.664673,106.780506,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00090,.PNG,1071,False
3,entrenamiento,img001-00122.png,0,128,128,L,16384,212.742249,92.932467,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00122,.PNG,609,False
4,entrenamiento,img001-00149.png,0,128,128,L,16384,194.080872,107.077416,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00149,.PNG,1074,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,prueba,img004-00518.png,3,128,128,L,16384,210.496216,95.506976,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00518,.PNG,1030,False
296,prueba,img004-00871.png,3,128,128,L,16384,221.958008,84.390821,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00871,.PNG,1073,False
297,prueba,img004-00918.png,3,128,128,L,16384,223.119934,82.675121,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00918,.PNG,1074,False
298,prueba,img004-00923.png,3,128,128,L,16384,232.317871,70.714580,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00923,.PNG,1105,False


In [5]:
#@title Cargar información sobre la Clase detectada por Modelos de Detección de Objetos de TF-Hub

# parámetros de información a cargar
incluir_clase_ObjectDetection = True #@param {type:"boolean"}
#@markdown Seleccionar modelos a aplicar:
aplicar_modelo_MobileNet = True #@param {type:"boolean"}
aplicar_modelo_ResNet = False #@param {type:"boolean"}
aplicar_modelo_Inception = False #@param {type:"boolean"}
aplicar_modelo_EfficientNet = False #@param {type:"boolean"}
#@markdown (Nota: tener en cuenta que esta ejecución puede demorar)


def cargarModelo(modeloUsar):
  # URLs donde se encuentra la info del modelo Inception v3
  if modeloUsar == "Inception":
    classifier_url = "https://tfhub.dev/google/tf2-preview/inception_v3/classification/4"
    IMAGE_SHAPE = (299, 299)
  elif modeloUsar == "ResNet":
    classifier_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/classification/4"
    IMAGE_SHAPE = (224, 224)
  elif modeloUsar == "EfficientNet":
    classifier_url = "https://tfhub.dev/tensorflow/efficientnet/b0/classification/1"
    IMAGE_SHAPE = (224, 224)
  else: #modeloUsar == "MobileNet":
    classifier_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
    IMAGE_SHAPE = (224, 224)
  # carga el módulo a usar
  base_model = hub.KerasLayer(classifier_url, trainable=True, arguments=dict(batch_norm_momentum=0.997))
  classifier = tf.keras.Sequential([
      # Capa de entrada
      tf.keras.layers.InputLayer(input_shape=IMAGE_SHAPE+(3,)),
      # Envolver la capa del modelo base en una capa Lambda
      tf.keras.layers.Lambda(lambda x: base_model(x)),
      ])
  #classifier = tf.keras.Sequential([ hub.KerasLayer(classifier_url,  input_shape=IMAGE_SHAPE+(3,)) ])
  print("    - Modelo ", modeloUsar, "cargado")
  return [modeloUsar, IMAGE_SHAPE, classifier]

def procesarImage(imagFN, imageSize, classifier):
  # abre y prepara la imagen
  image = tf.keras.preprocessing.image.load_img(imagFN, target_size=imageSize)
  input_arr = tf.keras.preprocessing.image.img_to_array(image)
  input_arr_batch = np.array([input_arr])  # Convert single image to a batch.
  # obtiene la clase detectada por el modelo
  predictions = classifier.predict(input_arr_batch, verbose=0)
  predicted_class_name = model_imagenet_labels[np.argmax(predictions, axis=-1)]
  return predicted_class_name[0]

if incluir_clase_ObjectDetection and ("ndf" in globals()) and (ndf is not None):

  # se fija si ya se cargaron los modelos
  if ("modelsObjDet" not in globals()) or (modelsObjDet is None) or (len(modelsObjDet)==0):
    print("> Instalando el paquete para TF-Hub: ")
    print("  (se pueden ignorar los warnings)")
    # Instalar el paquete
    try:
      # %tensorflow_version only exists in Colab.
      !pip install -q tf-nightly
    except Exception:
      pass

    #!pip install -q -U tf-hub-nightly
    #!pip install -q tfds-nightly

    #carga los modelos a usar
    print("\n> Cargando modelos de detección de objetos:")
    modelsObjDet = []
    if aplicar_modelo_MobileNet:
      modelsObjDet.append( cargarModelo("MobileNet") )
    if aplicar_modelo_Inception:
      modelsObjDet.append( cargarModelo("Inception") )
    if aplicar_modelo_ResNet:
      modelsObjDet.append( cargarModelo("ResNet") )
    if aplicar_modelo_EfficientNet:
      modelsObjDet.append( cargarModelo("EfficientNet") )
  else:
    print("Usando ", len(modelsObjDet), " modelos de detección de objetos ya cargados.")

  # se fija si las clases ya están defindias
  if ("model_imagenet_labels" not in globals()) or (model_imagenet_labels is None):
    # baja la lista de clases originales que maneja el modelo
    model_labels_path = tf.keras.utils.get_file('ImageNetLabels.txt',
                                                'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
    model_imagenet_labels = np.array(open(model_labels_path).read().splitlines())

  # procesa las imágenes del dataset
  print("\n> Procesando imágenes:")
  for mod in modelsObjDet:
    print("    usando modelo ", mod[0])
    resModel = []
    # procesa las imágenes
    for index, row in ndf.iterrows():
      resModel.append( procesarImage(row["Archivo_Path"]+'/'+row["Imagen"], mod[1], mod[2]) )
    # agrega información al dataframe
    ndf["Clase_"+mod[0]] = resModel

  print("\n> Información agregadada por Modelo Detección de Objetos: ")
  display( ndf )

> Instalando el paquete para TF-Hub: 
  (se pueden ignorar los warnings)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 655.4/655.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.

> Cargando modelos de detección de objetos:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


    - Modelo  MobileNet cargado
10484/10484 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

> Procesando imágenes:
    usando modelo  MobileNet

> Información agregadada por Modelo Detección de Objetos: 


,TipoUso,Imagen,Clase,Img_Ancho,Img_Alto,Img_Modo,Pixeles_L_Total,Pixeles_L_Promedio,Pixeles_L_Desvio,Archivo_Path,Archivo_Nombre,Archivo_Extension,Archivo_TamBytes,Archivo_EsDA,Clase_MobileNet
0,entrenamiento,img001-00052.png,0,128,128,L,16384,197.708801,104.907928,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00052,.PNG,1066,False,loupe
1,entrenamiento,img001-00077.png,0,128,128,L,16384,158.532166,122.573417,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00077,.PNG,1059,False,spotlight
2,entrenamiento,img001-00090.png,0,128,128,L,16384,194.664673,106.780506,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00090,.PNG,1071,False,reflex camera
3,entrenamiento,img001-00122.png,0,128,128,L,16384,212.742249,92.932467,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00122,.PNG,609,False,ballpoint
4,entrenamiento,img001-00149.png,0,128,128,L,16384,194.080872,107.077416,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0,img001-00149,.PNG,1074,False,bib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,prueba,img004-00518.png,3,128,128,L,16384,210.496216,95.506976,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00518,.PNG,1030,False,hook
296,prueba,img004-00871.png,3,128,128,L,16384,221.958008,84.390821,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00871,.PNG,1073,False,hook
297,prueba,img004-00918.png,3,128,128,L,16384,223.119934,82.675121,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00918,.PNG,1074,False,hook
298,prueba,img004-00923.png,3,128,128,L,16384,232.317871,70.714580,gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3,img004-00923,.PNG,1105,False,whistle


In [6]:
#@title Mostrar Estadísticas Generales de la Información cargada de las Imágenes { run: "auto" }

# muestra las estadísticas
generar_estadisticas_detalladas(ndf, "> Estadísticas de los datos cargados")



 > Estadísticas de los datos cargados : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
TipoUso,object,{ TipoUso ; entrenamiento 240 ; prueba 60 },-,0,0
Imagen,object,{ Imagen ; img001-00052.png 1 ; img003-00424.png 1 ; img003-00154.png 1 ; img003-00331.png 1 ; img003-00046.png 1 ; .. ; img009-00218.png 1 ; img009-00142.png 1 ; img009-00133.png 1 ; img009-00047.png 1 ; img004-01015.png 1 },-,0,0
Clase,object,{ Clase ; 0 30 ; 4 30 ; 5 30 ; 6 30 ; 8 30 ; 1 30 ; 9 30 ; 7 30 ; 2 30 ; 3 30 },-,0,0
Img_Ancho,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Alto,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Modo,object,{ Img_Modo ; L 300 },-,0,0
Pixeles_L_Total,int64,[ 16384.00 ; 16384.00 ],16384.000 ± 0.000,0,0
Pixeles_L_Promedio,float64,[ 153.19 ; 252.01 ],214.285 ± 14.834,0,0
Pixeles_L_Desvio,float64,[ 27.44 ; 124.03 ],89.722 ± 14.199,0,0
Archivo_Path,object,{ Archivo_Path ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/5 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/6 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/8 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/1 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/9 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/7 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/2 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/3 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/4 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/1 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/2 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/7 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/9 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/0 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/8 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/6 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/5 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/4 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3 6 },-,0,0


Tiene 15 atributos y 300 ejemplos.




#Preparación de los Datos:

In [7]:
#@title Seleccionar columnas de los datos (OPCIONAL) { run: "auto" }

seleccionar_columnas_datos = False #@param {type:"boolean"}

if seleccionar_columnas_datos:

  # copia de los datos
  copiaDF = ndf.copy()

  # carga las columnas numéricas y no numéricas
  todasCols = list(ndf.columns)

  # combo selección de columnas
  selColumnas = widgets.SelectMultiple(
      options=todasCols,
      value=list(todasCols),
      description='Atributos seleccionados:',
      rows=len(todasCols),
      layout=Layout(display="flex")
  )

  ui = widgets.HBox([selColumnas])


  def cambiaSeleccion(cols):
    global ndf
    # convierte la selección a un array
    arCols = np.array(cols)
    # ordena los datos
    print("\n> Se seleccionan los atributos: ", arCols)
    ndf = copiaDF[arCols]
    print("\n> 10 primeros ejemplos de los datos con menos columnas: ")
    display(ndf.head())

  out = widgets.interactive_output(cambiaSeleccion, {'cols': selColumnas})
  display(ui, out)

else:
  print("No se selecciona columnas de los datos.")


No se selecciona columnas de los datos.


In [8]:
#@title Definir Atributo Clase (OPCIONAL) { run: "auto" }

#@markdown Nombre del atributo clase / objetivo:
atributo_clase = 'Clase' #@param {type:"string"}
mostrar_estadisticas_por_valor_clase = True #@param {type:"boolean"}

if atributo_clase != "":

  # controla que exista el atributo_clase
  if atributo_clase not in ndf:
    raise ValueError("El atributo '" + atributo_clase + "' no existe en los datos cargados!!!")

  # mostrar nuevas estadísticas por Clase
  if mostrar_estadisticas_por_valor_clase:
    for val in  ndf[atributo_clase].fillna("-NAN-").unique():
      auxDF = ndf[ ndf[atributo_clase].fillna("-NAN-") == val]
      auxDF = auxDF.drop(labels=atributo_clase, axis=1)
      generar_estadisticas_detalladas(auxDF, "- Estadísticas por valor de clase '" + str(val) + "'" )

else:
  print("No se ha definido el nombre del atributo clase.")


 - Estadísticas por valor de clase '0' : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
TipoUso,object,{ TipoUso ; entrenamiento 24 ; prueba 6 },-,0,0
Imagen,object,{ Imagen ; img001-00052.png 1 ; img001-00077.png 1 ; img001-00503.png 1 ; img001-00272.png 1 ; img001-00252.png 1 ; img001-00070.png 1 ; img001-00020.png 1 ; img001-00992.png 1 ; img001-00968.png 1 ; img001-00953.png 1 ; img001-00941.png 1 ; img001-00930.png 1 ; img001-00920.png 1 ; img001-00901.png 1 ; img001-00847.png 1 ; img001-00827.png 1 ; img001-00774.png 1 ; img001-00740.png 1 ; img001-00709.png 1 ; img001-00465.png 1 ; img001-00409.png 1 ; img001-00416.png 1 ; img001-00378.png 1 ; img001-00354.png 1 ; img001-00295.png 1 ; img001-00214.png 1 ; img001-00149.png 1 ; img001-00122.png 1 ; img001-00090.png 1 ; img001-00994.png 1 },-,0,0
Img_Ancho,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Alto,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Modo,object,{ Img_Modo ; L 30 },-,0,0
Pixeles_L_Total,int64,[ 16384.00 ; 16384.00 ],16384.000 ± 0.000,0,0
Pixeles_L_Promedio,float64,[ 153.19 ; 237.55 ],203.007 ± 17.858,0,0
Pixeles_L_Desvio,float64,[ 61.85 ; 124.03 ],98.794 ± 12.709,0,0
Archivo_Path,object,{ Archivo_Path ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/0 6 },-,0,0
Archivo_Nombre,object,{ Archivo_Nombre ; img001-00052 1 ; img001-00077 1 ; img001-00503 1 ; img001-00272 1 ; img001-00252 1 ; img001-00070 1 ; img001-00020 1 ; img001-00992 1 ; img001-00968 1 ; img001-00953 1 ; img001-00941 1 ; img001-00930 1 ; img001-00920 1 ; img001-00901 1 ; img001-00847 1 ; img001-00827 1 ; img001-00774 1 ; img001-00740 1 ; img001-00709 1 ; img001-00465 1 ; img001-00409 1 ; img001-00416 1 ; img001-00378 1 ; img001-00354 1 ; img001-00295 1 ; img001-00214 1 ; img001-00149 1 ; img001-00122 1 ; img001-00090 1 ; img001-00994 1 },-,0,0


Tiene 14 atributos y 30 ejemplos.



 - Estadísticas por valor de clase '4' : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
TipoUso,object,{ TipoUso ; entrenamiento 24 ; prueba 6 },-,0,0
Imagen,object,{ Imagen ; img005-00783.png 1 ; img005-00675.png 1 ; img005-00750.png 1 ; img005-00488.png 1 ; img005-00379.png 1 ; img005-00255.png 1 ; img005-00955.png 1 ; img005-00456.png 1 ; img005-00423.png 1 ; img005-00400.png 1 ; img005-00751.png 1 ; img005-00551.png 1 ; img005-00567.png 1 ; img005-00627.png 1 ; img005-00583.png 1 ; img005-00916.png 1 ; img005-00437.png 1 ; img005-00710.png 1 ; img005-00514.png 1 ; img005-00090.png 1 ; img005-00639.png 1 ; img005-00672.png 1 ; img005-00170.png 1 ; img005-00104.png 1 ; img005-00409.png 1 ; img005-00512.png 1 ; img005-00742.png 1 ; img005-00610.png 1 ; img005-00734.png 1 ; img005-00822.png 1 },-,0,0
Img_Ancho,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Alto,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Modo,object,{ Img_Modo ; L 30 },-,0,0
Pixeles_L_Total,int64,[ 16384.00 ; 16384.00 ],16384.000 ± 0.000,0,0
Pixeles_L_Promedio,float64,[ 191.62 ; 237.57 ],216.700 ± 10.918,0,0
Pixeles_L_Desvio,float64,[ 64.35 ; 109.22 ],88.425 ± 10.994,0,0
Archivo_Path,object,{ Archivo_Path ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/4 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/4 6 },-,0,0
Archivo_Nombre,object,{ Archivo_Nombre ; img005-00783 1 ; img005-00675 1 ; img005-00750 1 ; img005-00488 1 ; img005-00379 1 ; img005-00255 1 ; img005-00955 1 ; img005-00456 1 ; img005-00423 1 ; img005-00400 1 ; img005-00751 1 ; img005-00551 1 ; img005-00567 1 ; img005-00627 1 ; img005-00583 1 ; img005-00916 1 ; img005-00437 1 ; img005-00710 1 ; img005-00514 1 ; img005-00090 1 ; img005-00639 1 ; img005-00672 1 ; img005-00170 1 ; img005-00104 1 ; img005-00409 1 ; img005-00512 1 ; img005-00742 1 ; img005-00610 1 ; img005-00734 1 ; img005-00822 1 },-,0,0


Tiene 14 atributos y 30 ejemplos.



 - Estadísticas por valor de clase '5' : 


...

Tiene 14 atributos y 30 ejemplos.



 - Estadísticas por valor de clase '2' : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
TipoUso,object,{ TipoUso ; entrenamiento 24 ; prueba 6 },-,0,0
Imagen,object,{ Imagen ; img003-00337.png 1 ; img003-00222.png 1 ; img003-00921.png 1 ; img003-00931.png 1 ; img003-00893.png 1 ; img003-00266.png 1 ; img003-00742.png 1 ; img003-00351.png 1 ; img003-00062.png 1 ; img003-00324.png 1 ; img003-00471.png 1 ; img003-00404.png 1 ; img003-00456.png 1 ; img003-00792.png 1 ; img003-00467.png 1 ; img003-00997.png 1 ; img003-00424.png 1 ; img003-00486.png 1 ; img003-00154.png 1 ; img003-00331.png 1 ; img003-00046.png 1 ; img003-00108.png 1 ; img003-00666.png 1 ; img003-00617.png 1 ; img003-00063.png 1 ; img003-00201.png 1 ; img003-00361.png 1 ; img003-00775.png 1 ; img003-00067.png 1 ; img003-00968.png 1 },-,0,0
Img_Ancho,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Alto,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Modo,object,{ Img_Modo ; L 30 },-,0,0
Pixeles_L_Total,int64,[ 16384.00 ; 16384.00 ],16384.000 ± 0.000,0,0
Pixeles_L_Promedio,float64,[ 179.07 ; 233.09 ],212.623 ± 12.444,0,0
Pixeles_L_Desvio,float64,[ 70.03 ; 115.69 ],92.182 ± 11.560,0,0
Archivo_Path,object,{ Archivo_Path ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/2 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/2 6 },-,0,0
Archivo_Nombre,object,{ Archivo_Nombre ; img003-00337 1 ; img003-00222 1 ; img003-00921 1 ; img003-00931 1 ; img003-00893 1 ; img003-00266 1 ; img003-00742 1 ; img003-00351 1 ; img003-00062 1 ; img003-00324 1 ; img003-00471 1 ; img003-00404 1 ; img003-00456 1 ; img003-00792 1 ; img003-00467 1 ; img003-00997 1 ; img003-00424 1 ; img003-00486 1 ; img003-00154 1 ; img003-00331 1 ; img003-00046 1 ; img003-00108 1 ; img003-00666 1 ; img003-00617 1 ; img003-00063 1 ; img003-00201 1 ; img003-00361 1 ; img003-00775 1 ; img003-00067 1 ; img003-00968 1 },-,0,0


Tiene 14 atributos y 30 ejemplos.



 - Estadísticas por valor de clase '3' : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
TipoUso,object,{ TipoUso ; entrenamiento 24 ; prueba 6 },-,0,0
Imagen,object,{ Imagen ; img004-00006.png 1 ; img004-00044.png 1 ; img004-00923.png 1 ; img004-00918.png 1 ; img004-00871.png 1 ; img004-00518.png 1 ; img004-00282.png 1 ; img004-00981.png 1 ; img004-00937.png 1 ; img004-00896.png 1 ; img004-00877.png 1 ; img004-00851.png 1 ; img004-00824.png 1 ; img004-00738.png 1 ; img004-00665.png 1 ; img004-00657.png 1 ; img004-00609.png 1 ; img004-00535.png 1 ; img004-00528.png 1 ; img004-00491.png 1 ; img004-00406.png 1 ; img004-00381.png 1 ; img004-00346.png 1 ; img004-00288.png 1 ; img004-00263.png 1 ; img004-00175.png 1 ; img004-00172.png 1 ; img004-00165.png 1 ; img004-00090.png 1 ; img004-01015.png 1 },-,0,0
Img_Ancho,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Alto,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Modo,object,{ Img_Modo ; L 30 },-,0,0
Pixeles_L_Total,int64,[ 16384.00 ; 16384.00 ],16384.000 ± 0.000,0,0
Pixeles_L_Promedio,float64,[ 191.47 ; 252.01 ],217.679 ± 12.526,0,0
Pixeles_L_Desvio,float64,[ 27.44 ; 109.00 ],86.571 ± 15.741,0,0
Archivo_Path,object,{ Archivo_Path ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/3 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3 6 },-,0,0
Archivo_Nombre,object,{ Archivo_Nombre ; img004-00006 1 ; img004-00044 1 ; img004-00923 1 ; img004-00918 1 ; img004-00871 1 ; img004-00518 1 ; img004-00282 1 ; img004-00981 1 ; img004-00937 1 ; img004-00896 1 ; img004-00877 1 ; img004-00851 1 ; img004-00824 1 ; img004-00738 1 ; img004-00665 1 ; img004-00657 1 ; img004-00609 1 ; img004-00535 1 ; img004-00528 1 ; img004-00491 1 ; img004-00406 1 ; img004-00381 1 ; img004-00346 1 ; img004-00288 1 ; img004-00263 1 ; img004-00175 1 ; img004-00172 1 ; img004-00165 1 ; img004-00090 1 ; img004-01015 1 },-,0,0


Tiene 14 atributos y 30 ejemplos.




In [9]:
#@title Ordenar los datos (OPCIONAL) { run: "auto" }

ordenar_valores_datos = False #@param {type:"boolean"}

if ordenar_valores_datos:

  # carga las columnas numéricas y no numéricas
  todasCols = list(ndf.columns)

  # combo selección de columnas
  selColumnas = widgets.SelectMultiple(
      options=todasCols,
      value=list(todasCols),
      description='Atributos seleccionados:',
      rows=len(todasCols),
      layout=Layout(display="flex")
  )

  ui = widgets.HBox([selColumnas])


  def cambiaSeleccion(cols):
    global ndf
    # convierte la selección a un array
    arCols = np.array(cols)
    # ordena los datos
    print("\n> Ordenando por atributos: ", arCols)
    ndf = ndf.sort_values(by=list(arCols))
    print("\n> 10 primeros ejemplos de los datos ordenados: ")
    display(ndf.head())

  out = widgets.interactive_output(cambiaSeleccion, {'cols': selColumnas})
  display(ui, out)

else:
  print("No se ordenan los datos.")


No se ordenan los datos.


In [10]:
#@title Convertir atributos no numéricos a numéricos (OPCIONAL) { run: "auto" }

convertir_valores_noNumericos_a_Numericos = False #@param {type:"boolean"}
#@markdown (Nota: si está definido el atributo clase / objetivo se mantiene como no numérico)

# convierte a valores numéricos
if convertir_valores_noNumericos_a_Numericos:
  ndf = convColsNumericas(ndf, [atributo_clase])

  # mostrar estadísticas generales
  generar_estadisticas_detalladas(ndf, "> Nuevas Estadísticas Generales valores convertidos")
else:
  print("No se aplica la conversión a valores numéricos.")

No se aplica la conversión a valores numéricos.


In [11]:
#@title Controlar valores duplicados (OPCIONAL) { run: "auto" }

#@markdown Muesta ejemplos con los valores iguales:
detectar_ejemplos_duplicados = True #@param {type:"boolean"}

if detectar_ejemplos_duplicados:

  # carga las columnas numéricas y no numéricas
  todasCols = list(ndf.columns)

  # combo selección de columnas
  selColumnas = widgets.SelectMultiple(
      options=todasCols,
      value=list(todasCols),
      description='Atributos seleccionados:',
      rows=len(todasCols),
      layout=Layout(display="flex")
  )

  ui = widgets.HBox([selColumnas])

  dups = None

  # Botones de acciones
  def mostrarDups(b):
    with output:
      clear_output()
      global dups, ndf
      if dups is not None and len(ndf[dups])>0:
        display( ndf[dups] )
      else:
        print("No hay duplicados para mostrar!")
  def mostrarEstDups(b):
    with output:
      clear_output()
      global dups, ndf
      if dups is not None and len(ndf[dups])>0:
        generar_estadisticas_detalladas(ndf[dups], "Estadística de ejemplos duplicados")
      else:
        print("No hay duplicados para mostrar!")
  def eliminarDups(b):
    with output:
      clear_output()
      global dups, ndf
      if dups is not None and len(ndf[dups])>0:
        ndf = ndf[~dups]
        cambiaSeleccionDup(selColumnas.value)
        print(" Ejemplos duplicados eliminados.")
      else:
        print("No hay duplicados para eliminar!")


  layoutButton = widgets.Layout(width='auto', height='40px') #set width and height
  buttonMD = widgets.Button(description="Mostrar Datos Duplicados",
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            layout = layoutButton)
  buttonMD.on_click(mostrarDups)
  buttonMeD = widgets.Button(description="Mostrar Estadísticas Duplicados",
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            layout = layoutButton)
  buttonMeD.on_click(mostrarEstDups)
  buttonED = widgets.Button(description="Eliminar Duplicados",
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            layout = layoutButton)
  buttonED.on_click(eliminarDups)
  output = widgets.Output()


  def cambiaSeleccionDup(cols):
    global dups
    # convierte la selección a un array
    arCols = np.array(cols)
    print("\n> Resultados de controlar ejemplos duplicados con atributos seleccionados: ")
    # detecta los ejemplos duplicados
    # (al primero le pone 'False' y los siguientes iguales se marcan con 'True')
    dups = ndf.duplicated(subset=arCols, keep="first")
    # muestra las cantidades
    cantDup = len(ndf[dups])
    cantNoDup = len(ndf[~dups])
    print("   - Cantidad de ejemplos con todos los valores duplicados = ", cantDup, "(", round((cantDup/(cantDup+cantNoDup))*100,1), "%)")
    print("   + Cantidad de ejemplos con valores diferentes = ", cantNoDup, "(", round((cantNoDup/(cantDup+cantNoDup))*100,1), "%)")
    print("   = Cantidad total de ejemplos = ", (cantDup+cantNoDup))

    if cantDup > 0:
      # si hay duplicados
      print("\n> Posibles acciones a realizar:")
      display(buttonMD, buttonMeD, buttonED, output)


  out = widgets.interactive_output(cambiaSeleccionDup, {'cols': selColumnas})
  display(ui, out)

else:
  print("No se controlan datos duplicados.")


Output()

In [12]:
#@title Normalizar datos numéricos (OPCIONAL) { run: "auto" }

aplica_normalizacion = False #@param {type:"boolean"}
#@markdown Si se aplica, seleccione el tipo de método de normalización a aplicar:
tipo_normalizacion = "MinMax Scaler" #@param ["Standard Scaler", "MinMax Scaler", "MaxAbs Scaler", "Robust Scaler"]

if aplica_normalizacion:

  print("10 primeros ejemplos antes de normalizar: ")
  display( ndf.head() )

  from sklearn import preprocessing

  # elegir el método de normalización
  if tipo_normalizacion == "Standard Scaler":
    scaler = preprocessing.StandardScaler()
  elif tipo_normalizacion == "MinMax Scaler":
    scaler = preprocessing.MinMaxScaler()
  elif tipo_normalizacion == "MaxMax Scaler":
    scaler = preprocessing.MaxAbsScaler()
  elif tipo_normalizacion == "Robust Scaler":
    scaler = preprocessing.RobustScaler()

  # aplica la normalización sólo en columnas de datos
  for col in ndf.columns:
    if ndf[col].dtypes != "object":
      ndf[col] = scaler.fit_transform(ndf[col].values.reshape(-1, 1))

  print("\n\n10 primeros ejemplos después de normalizar: ")
  display( ndf.head() )

  # muestra las estadísticas del normalizado
  generar_estadisticas_detalladas(ndf, "> Estadísticas de los datos normalizados")
else:
  print("No se aplica la normalización de los datos.")

No se aplica la normalización de los datos.


In [13]:
#@title Mostrar Estadísticas Generales de Datos Preparados { run: "auto" }

# muestra las estadísticas
generar_estadisticas_detalladas(ndf, "> Estadísticas de los datos preparados")



 > Estadísticas de los datos preparados : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
TipoUso,object,{ TipoUso ; entrenamiento 240 ; prueba 60 },-,0,0
Imagen,object,{ Imagen ; img001-00052.png 1 ; img003-00424.png 1 ; img003-00154.png 1 ; img003-00331.png 1 ; img003-00046.png 1 ; .. ; img009-00218.png 1 ; img009-00142.png 1 ; img009-00133.png 1 ; img009-00047.png 1 ; img004-01015.png 1 },-,0,0
Clase,object,{ Clase ; 0 30 ; 4 30 ; 5 30 ; 6 30 ; 8 30 ; 1 30 ; 9 30 ; 7 30 ; 2 30 ; 3 30 },-,0,0
Img_Ancho,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Alto,int64,[ 128.00 ; 128.00 ],128.000 ± 0.000,0,0
Img_Modo,object,{ Img_Modo ; L 300 },-,0,0
Pixeles_L_Total,int64,[ 16384.00 ; 16384.00 ],16384.000 ± 0.000,0,0
Pixeles_L_Promedio,float64,[ 153.19 ; 252.01 ],214.285 ± 14.834,0,0
Pixeles_L_Desvio,float64,[ 27.44 ; 124.03 ],89.722 ± 14.199,0,0
Archivo_Path,object,{ Archivo_Path ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/0 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/5 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/6 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/8 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/1 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/9 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/7 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/2 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/3 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train/4 24 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/1 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/2 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/7 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/9 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/0 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/8 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/6 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/5 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/4 6 ; gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test/3 6 },-,0,0


Tiene 15 atributos y 300 ejemplos.




In [14]:
#@title Exportar Datos Preparados (OPCIONAL) { run: "auto" }

exportar_datos_preparados = False #@param {type:"boolean"}
#@markdown  Archivo de datos a grabar:
nuevo_archivo_datos = 'salida.csv'  #@param {type:"string"}

if exportar_datos_preparados and nuevo_archivo_datos!="":
  # controla que no exista el archivo
  if os.path.isfile(path + nuevo_archivo_datos):
    print("Error: el archivo ", path + nuevo_archivo_datos, " ya existe!")
  else:
    # exporta los datos preparados actuales como CSV
    ndf.to_csv(path + nuevo_archivo_datos, index=False)
    print("Nuevo archivo de datos creado: ", path + nuevo_archivo_datos)


# Gráficos Estadísticos:

In [15]:
#@title Generar gráfico de Quartile o BoxPlot { run: "auto" }

def genGraph_BoxPlot(ndf, titulo):
  fig = plt.figure(figsize=(15,8))
  legValues = []
  colValues = []
  for col in ndf.columns:
    if ndf[col].dtypes != "object":
      legValues.append( col )
      colValues.append( ndf[col].fillna(0.001) )
  if len(colValues)>0:
    ax = fig.add_axes( [0, 0, 0.8, 0.8] )
    ax.boxplot( colValues )
    ax.set_xticklabels( legValues )
  plt.grid(color='lightgrey', which='major', axis='y', linestyle='solid', linewidth=0.3)
  plt.title(titulo)
  plt.show()

# carga las columnas numéricas y no numéricas
colValues, _ = devolNombreColumnas(ndf)

selColumnas = widgets.SelectMultiple(
    options=colValues,
    value=list(colValues),
    description='Atributos a mostrar:')

# agrega CB para determinar si muestra por clase
generaXclase = widgets.Checkbox(
    value=False,
    description='Generar gráfico por Valor Clase',
    disabled=False,
    indent=False
)

ui = widgets.VBox([selColumnas, generaXclase])

def cambiaSeleccion(cols, grafico_por_valor_clase):
  # convierte la selección a un array
  arCols = np.array(cols)
  # genera los gráficos
  tit = "Gráfico Estadístico de Quartile / BoxPlot"
  if grafico_por_valor_clase:
    for val in  ndf[atributo_clase].unique():
      auxDF = ndf[ ndf[atributo_clase] == val]
      auxDF = auxDF.drop(labels=atributo_clase, axis=1)
      genGraph_BoxPlot(auxDF[arCols], tit + " para valor de clase '" + str(val) + "'" )
  else:
    genGraph_BoxPlot(ndf[arCols], tit)


out = widgets.interactive_output(cambiaSeleccion, {'cols': selColumnas, 'grafico_por_valor_clase': generaXclase})
display(ui, out)



Output()

In [16]:
#@title Generar gráfico de Deviation  { run: "auto" }

def genGraph_Deviation(oriDF, titulo, porClase):
  fig = plt.figure(figsize=(15,8))

  listDF = []
  legValues = []
  if porClase:
    # genera sub-DF por valor de clase
    for val in oriDF[atributo_clase].unique():
      auxDF = oriDF[ oriDF[atributo_clase] == val]
      auxDF = auxDF.drop(labels=atributo_clase, axis=1)
      listDF.append( auxDF.fillna(0.001) )
      legValues.append( atributo_clase + " " + str(val))
  else:
    # usa todos los datos como sub-DF
    listDF.append( oriDF )

  # genera línea por sub-DF
  for ndf in listDF:
    xLabel = []
    meanValues = []
    stdValues = []
    for col in ndf.columns:
      if ndf[col].dtypes != "object":
        xLabel.append( col )
        meanValues.append( ndf[col].mean() )
        stdValues.append( ndf[col].std() )
    plt.errorbar(xLabel, meanValues, yerr=stdValues)

  plt.grid(color='lightgrey', which='major', axis='y', linestyle='solid', linewidth=0.3)
  plt.title(titulo)
  if len(legValues) > 0:
    plt.legend(legValues, loc='best')
  plt.show()


# carga las columnas numéricas y no numéricas
colValues, _ = devolNombreColumnas(ndf)

selColumnas = widgets.SelectMultiple(
    options=colValues,
    value=list(colValues),
    description='Atributos a mostrar:')

# agrega CB para determinar si muestra por clase
generaXclase = widgets.Checkbox(
    value=False,
    description='Generar gráfico por Valor Clase',
    disabled=False,
    indent=False
)

ui = widgets.VBox([selColumnas, generaXclase])

def cambiaSeleccion(cols, grafico_por_valor_clase):
  # convierte la selección a un array
  if grafico_por_valor_clase:
    # agrega el atributo clase para mostrar
    cols = list(cols)
    cols.append(atributo_clase)
  arCols = np.array(cols)
  # genera  gráfico
  tit = "Gráfico Estadístico de Deviation"
  genGraph_Deviation(ndf[arCols], tit, grafico_por_valor_clase)


out = widgets.interactive_output(cambiaSeleccion, {'cols': selColumnas, 'grafico_por_valor_clase': generaXclase})
display(ui, out)

Output()

In [17]:
#@title Generar gráfico de Histogram { run: "auto" }

cantidad_bins = 10 #@param {type:"integer"}

if cantidad_bins <= 1:
  cantidad_bins = 1

def genGraph_Histogram(ndf, titulo):
  fig = plt.figure(figsize=(15,8))
  legValues = []
  for col in ndf.columns:
    if ndf[col].dtypes != "object":
      plt.hist( ndf[col].fillna(0.001), bins = cantidad_bins )
      legValues.append( col )
  plt.grid(color='lightgrey', which='major', axis='x', linestyle='solid', linewidth=0.2)
  plt.grid(color='lightgrey', which='major', axis='y', linestyle='solid', linewidth=0.3)
  plt.title(titulo)
  plt.xlabel('Valor')
  plt.ylabel('Frencuencia')
  plt.legend(legValues, loc='best')
  plt.show()

# carga las columnas numéricas y no numéricas
colValues, _ = devolNombreColumnas(ndf)

selColumnas = widgets.SelectMultiple(
    options=colValues,
    value=list(colValues),
    description='Atributos a mostrar:')

# agrega CB para determinar si muestra por clase
generaXclase = widgets.Checkbox(
    value=False,
    description='Generar gráfico por Valor Clase',
    disabled=False,
    indent=False
)

ui = widgets.VBox([selColumnas, generaXclase])

def cambiaSeleccion(cols, grafico_por_valor_clase):
  # convierte la selección a un array
  arCols = np.array(cols)
  # genera los gráficos
  tit = "Gráfico Estadístico de Histograma"
  if grafico_por_valor_clase:
    for val in  ndf[atributo_clase].fillna("-NAN-").unique():
      auxDF = ndf[ ndf[atributo_clase] == val]
      auxDF = auxDF.drop(labels=atributo_clase, axis=1)
      genGraph_Histogram(auxDF[arCols], tit + " para valor de clase '" + str(val) + "'" )
  else:
    genGraph_Histogram(ndf[arCols], tit)



out = widgets.interactive_output(cambiaSeleccion, {'cols': selColumnas, 'grafico_por_valor_clase': generaXclase})
display(ui, out)




Output()

In [18]:
#@title Generar gráfico de Scatter

# carga las columnas numéricas y no numéricas
colValues, colNoValues = devolNombreColumnas(ndf)
colNoValues.insert(0, "-")

# agrega los combos
comboColX = widgets.Dropdown(
    options=colValues,
    value=colValues[0],
    description='Columna X:',
    disabled=False,
)
if len(colValues)>1:
  valYsel = colValues[1]
else:
  valYsel = colValues[0]

comboColY = widgets.Dropdown(
    options=colValues,
    value=valYsel,
    description='Columna Y:',
    disabled=False,
)

comboColZ = widgets.Dropdown(
    options=colNoValues,
    value=colNoValues[0],
    description='Columna Z:',
    disabled=False,
)

ui = widgets.HBox([comboColX, comboColY, comboColZ])

def genGraph_Scatter(ndf, colx, coly, colz, titulo):
  if colx not in ndf or coly not in ndf or (colz!="" and colz not in ndf):
    if colx not in ndf:
      print("Nombre de columna X '" + colx + "' no existente!!")
    elif coly not in ndf:
      print("Nombre de columna Y '" + colx + "' no existente!!")
    elif (colz!="" and colz not in ndf):
      print("Nombre de columna Z '" + colx + "' no existente!!")
    return
  else:
    fig = plt.figure(figsize=(15,8))
    if colz == "":
      # gráfico general
      plt.scatter(ndf[colx], ndf[coly])
    else:
      # gráfico marcando colores por valor de colz
      legValues = []
      for val in  ndf[colz].unique():
        auxDF = ndf[ ndf[colz] == val]
        auxDF = auxDF.drop(labels=colz, axis=1)
        plt.scatter(auxDF[colx], auxDF[coly])
        legValues.append(val)
      plt.legend(legValues, loc='best')
    plt.xlabel(colx)
    plt.ylabel(coly)
    plt.grid(color='lightgrey', which='major', axis='y', linestyle='solid', linewidth=0.3)
    plt.grid(color='lightgrey', which='major', axis='x', linestyle='solid', linewidth=0.3)
    plt.show()

def cambiaSeleccion(x, y, z):
  # llama a la función
  # para generar el gráfico con las columnas seleccionadas
  tit = "Gráfico Estadístico de Scatter"
  if z == "-":
    z = ""
  genGraph_Scatter(ndf, x, y, z, tit)

out = widgets.interactive_output(cambiaSeleccion, {'x': comboColX, 'y': comboColY, 'z': comboColZ})
display(ui, out)

Output()

#Resultados de aplicar Algoritmos de Minería de Datos:

In [19]:
#@title Aplicar Descubrimiento de Reglas ( Decision Tree ) { run: "auto" }

# carga las columnas no numéricas
colValues, colNoValues = devolNombreColumnas(ndf)

if len(colNoValues)<1:
  raise ValueError("No se encuentra ningún atributo no numérico para utilizar como objetivo!!!")

selColumnas = widgets.SelectMultiple(
    options=colValues,
    value=list(colValues),
    description='Atr.Entrada:')

comboClase = widgets.Dropdown(
    options=colNoValues,
    value=(atributo_clase if atributo_clase!="" else colNoValues[0]),
    description='Atr.Clase:',
    disabled=False,
)

ui = widgets.HBox([selColumnas, comboClase])

def cambiaSeleccion(atributos_entrada, atributo_clase_tree):
    # convierte la selección a un array
    # agrega el atributo clase para mostrar
    cols = list(atributos_entrada)
    cols.append(atributo_clase_tree)
    arCols = np.array(cols)

    # obtiene los datos para entrenar el algoritmo
    X, Y, xColName = separarDatosXY(ndf[arCols], atributo_clase_tree, xSoloNros=True)

    # determina nombre de atributos y valores clase
    cols = xColName
    classvalues = np.unique(Y)

    # genera el árbol y lo muestra
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X, Y)

    # muestra el árbol
    plt.figure(figsize=(15,8))
    tree.plot_tree(clf,
                  feature_names=cols,
                  class_names=classvalues,
                  impurity=False,
                  rounded=True,
                  fontsize=11)
    plt.show()

out = widgets.interactive_output(cambiaSeleccion, {'atributos_entrada': selColumnas, 'atributo_clase_tree': comboClase})
display(ui, out)

Output()

In [20]:
#@title Aplicar Ponderación ( Naive Bayes )

# carga las columnas numéricas y no numéricas
colValues, colNoValues = devolNombreColumnas(ndf)

if len(colNoValues)<1:
  raise ValueError("No se encuentra ningún atributo no numérico para utilizar como objetivo!!!")

# agrega los combos
comboEnt = widgets.Dropdown(
    options=colValues,
    value=colValues[0],
    description='Atr.Entrada:',
    disabled=False,
)

comboClase = widgets.Dropdown(
    options=colNoValues,
    value=colNoValues[0],
    description='Atr.Clase:',
    disabled=False,
)

ui = widgets.HBox([comboEnt, comboClase])

def cambiaSeleccion(atributo_valor_naiveBayes, atributo_clase_naiveBayes):
  # genera el gráfico
  plt.figure(figsize=(15,8))
  legValues = []
  # procesa los valores del atributo clase
  for val in ndf[atributo_clase_naiveBayes].fillna("-NAN-").unique():
      # filtra los datos
      auxDF = ndf[ ndf[atributo_clase_naiveBayes] == val]
      # obtiene valores X, Y de clase y atributo
      Y = np.array(auxDF.pop(atributo_clase_naiveBayes).fillna("-NAN-"))
      X = np.array(auxDF[atributo_valor_naiveBayes].fillna(0.001))
      if len(X)>0 and len(Y)>0:
        legValues.append(val)
        # calcula la densidad
        # instantiate and fit the KDE model
        kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
        kde.fit(X[:, None])
        # score_samples returns the log of the probability density
        x_d = np.linspace(np.min(X), np.max(X), 1000)
        logprob = kde.score_samples(x_d[:, None])
        # genera área del atributo
        plt.fill_between(x_d, np.exp(logprob), alpha=0.5)
        plt.plot(X, np.full_like(X, -0.01), '|k', markeredgewidth=1)
  plt.xlabel("Valores")
  plt.ylabel("Frecuencia")
  plt.legend(legValues, loc='best')
  plt.grid(color='lightgrey', which='major', axis='y', linestyle='solid', linewidth=0.3)
  plt.show()

out = widgets.interactive_output(cambiaSeleccion, {'atributo_valor_naiveBayes': comboEnt, 'atributo_clase_naiveBayes': comboClase })
display(ui, out)


Output()

In [21]:
#@title Aplicar Clustering ( K-Means ) { run: "auto" }

#@markdown Cantidad de clústeres a generar:
cantidad_clusters = 3 #@param {type:"integer"}
#@markdown Nombre del atributo clase / objetivo a descartar en el clustering:
atributo_clase_descartar_clustering = "" #@param {type:"string"}

# como mínimo se tienen que generar dos clusters
if cantidad_clusters < 2:
  cantidad_clusters = 2

# controla que exista el atributo_clase
if atributo_clase_descartar_clustering != "" and atributo_clase_descartar_clustering not in ndf:
  raise ValueError("El atributo '" + atributo_clase_descartar_clustering + "' no existe en los datos cargados!!!")

# obtiene los datos para entrenar el algoritmo
X, Y, colValues = separarDatosXY(ndf, atributo_clase_descartar_clustering, xSoloNros=True)
colValues = list(colValues)

# Entrena K-Means
kmeans = KMeans(n_clusters=cantidad_clusters).fit(X)
centroids = kmeans.cluster_centers_
print("Centroides Generados: \n", centroids)
print("\n")

# Determina la salida
labels = kmeans.predict(X)

# agrega los combos
comboColX = widgets.Dropdown(
    options=colValues,
    value=colValues[0],
    description='Columna X:',
    disabled=False,
)
if len(colValues)>1:
  valYsel = colValues[1]
else:
  valYsel = colValues[0]

comboColY = widgets.Dropdown(
    options=colValues,
    value=valYsel,
    description='Columna Y:',
    disabled=False,
)

ui = widgets.HBox([comboColX, comboColY])

def cambiaSeleccion(x, y):
  # llama a la función
  # para generar el gráfico con las columnas seleccionadas
  plt.figure(figsize=(15,8))
  plt.scatter(X[:, colValues.index(x)], X[:, colValues.index(y)], c=labels)
  plt.show()

out = widgets.interactive_output(cambiaSeleccion, {'x': comboColX, 'y': comboColY})
display(ui, out)




Centroides Generados: 
 [[  128.           128.         16384.           216.41908981
     88.20479947   910.88034188     0.        ]
 [  128.           128.         16384.           210.34982843
     93.40801108  1142.32191781     0.        ]
 [  128.           128.         16384.           223.06403413
     79.97303394   485.13513514     0.        ]]




Output()